# Performance of SigmaCCS on the test set with different coordinates generated by ETKDG and MMFF94

In [1]:
import sys
sys.path.append("..")
from sigma.GraphData import *
from sigma.model import *
import pandas as pd
import random
from pandas import Series,DataFrame
import numpy as np
from tqdm import tqdm
import statistics

In [2]:
ifile = '../data/TestData.csv'
ParameterPath = '../parameter/parameter.pkl'
mfileh5 = '../model/model_new.h5'

In [3]:
res = random.sample(range(1, 100), 30)
def Generating_coordinates_different_seeds(smiles, adduct, ccs, a, All_Atoms, ps = AllChem.ETKDGv3(),):
    '''
    * Using ETKDG to generate 3D coordinates of molecules
    *
    * Attributes
    * ----------
    * smiles    : The SMILES string of the molecule
    * adduct    : Adduct of molecules
    * ccs       : CCS of molecules
    * All_Atoms : Element set (The type of element provided must cover all elements contained in the molecule)
    * ps        : ETKDG algorithm provided by RDkit
    *
    * Returns
    * -------
    * succ_smiles : SMILES of The molecules with 3D conformation can be successfully generated
    * succ_adduct : Adduct of The molecules with 3D conformation can be successfully generated
    * succ_ccs    : CCS of The molecules with 3D conformation can be successfully generated
    * Coordinate  : 3D coordinates of molecules
    '''
    succ_smiles = []
    succ_adduct = []
    succ_ccs    = []
    Coordinate  = []
    
    INDEX = -1
    for smi in smiles:
        INDEX += 1
        try:
            iMol = Chem.MolFromSmiles(smi)
            iMol = Chem.RemoveHs(iMol)
        except:
            continue;
        atoms = [atom.GetSymbol() for atom in iMol.GetAtoms()]
        bonds = [bond for bond in iMol.GetBonds()]
        # Is the number of atoms greater than 1
        if len(atoms) == 1 and len(bonds) <= 1:
            continue;
        # Determine whether the element is in all_ In atoms
        Elements_not_included = 0
        for atom in atoms:
            if atom not in All_Atoms:
                Elements_not_included = 1
        if Elements_not_included == 1:
            continue;
        # Adding H to a molecular object
        iMol3D = Chem.AddHs(iMol)
        
        # The 3D conformation of the generating molecule
        
        ps.randomSeed = a   #!!!!!!!!
        ps.maxAttempts = 1
        ps.numThreads = 0
        ps.useRandomCoords = True
        re = AllChem.EmbedMultipleConfs(iMol3D, numConfs = 1, params = ps)
        # Whether the conformation is successful or not
        if len(re) == 0:
            print('conformation is error')
            continue;
        # MMFF94
        re = AllChem.MMFFOptimizeMoleculeConfs(iMol3D,  numThreads = 0)

        This_mol_Coordinate = []
        for atom in iMol3D.GetAtoms():
            Coord = list(iMol3D.GetConformer().GetAtomPosition(atom.GetIdx()))
            This_mol_Coordinate.append(Coord)
        Coordinate.append(This_mol_Coordinate)
        
        succ_smiles.append(smi)
        succ_adduct.append(adduct[INDEX])
        succ_ccs.append(ccs[INDEX])
            
    return succ_smiles, succ_adduct, succ_ccs,  Coordinate

In [4]:
R2 = []
MeRE = []
randomSeeds = []

for ii in range(30):
    smiles, adduct, ccs = read_data(ifile)
    param = parameter.Parameter()
    with open(ParameterPath,'rb') as file:
        param  = pickle.loads(file.read())  
    a = res[ii]
    smiles, adduct, ccs, Coordinate = Generating_coordinates_different_seeds(smiles, adduct, ccs, a, param.All_Atoms)   ###
    
    for i in range(len(Coordinate)):
        Coordinate[i] = (np.array(Coordinate[i]) - param.Min_Coor) / (param.Max_Coor - param.Min_Coor)
    
    adj, features, edge_features = convertToGraph(smiles, Coordinate, param.All_Atoms)
    DataSet = MyDataset(features, adj, edge_features, ccs)
    
    ECC_Model = load_Model_from_file(mfileh5)
        
    re = predict(ECC_Model,param.adduct_SET,DataSet,adduct,)
    print("The number of molecules in the test set :", len(re), '\n')

    Sigma   = re
    CCS     = ccs
    SigmaPer = Metrics(CCS, Sigma)
    print("randomSeed :", a)
    print("R2 Score :", SigmaPer[0])
    print("Median Relative Error :", SigmaPer[1], '%')
    
    R2.append(SigmaPer[0])
    MeRE.append(SigmaPer[1])
    randomSeeds.append(a)

..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 558/559 [00:27<00:00, 20.37it/s]
The number of molecules in the test set : 559 

randomSeed : 45
R2 Score : 0.9937630410609587
Median Relative Error : 1.2043710031130037 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 558/559 [00:26<00:00, 20.76it/s]
The number of molecules in the test set : 559 

randomSeed : 34
R2 Score : 0.9938415869677824
Median Relative Error : 1.2026298794199095 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

...

Predictions: 100%|█████████████████████████████████████▉| 558/559 [00:24<00:00, 22.92it/s]
The number of molecules in the test set : 559 

randomSeed : 37
R2 Score : 0.9938509774176151
Median Relative Error : 1.1873798209848483 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 558/559 [00:10<00:00, 54.86it/s]
The number of molecules in the test set : 559 

randomSeed : 35
R2 Score : 0.9938795558837294
Median Relative Error : 1.2301139283978768 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 558/559 [00:09<00:00, 55.86it/s]
The number of molecules in the test set : 559 

randomSeed : 91
R2 Score : 0.9938147113891018
Median Relative Error : 1.1872586468987925 %


In [5]:
r2_mean = statistics.mean(R2)
r2_std = statistics.stdev(R2)

MeRE_mean = statistics.mean(MeRE)
MeRE_std = statistics.stdev(MeRE)

print("the mean value of R2 :", round(r2_mean, 4))
print("the standard deviation of R2 :", round(r2_std, 5))

print("the mean value of Median Relative Error :", round(MeRE_mean, 3), '%')
print("the standard deviation of Median Relative Error :", round(MeRE_std, 4), '%')

the mean value of R2 : 0.9938
the standard deviation of R2 : 8e-05
the mean value of Median Relative Error : 1.226 %
the standard deviation of Median Relative Error : 0.0269 %


In [6]:
for j in range(len(randomSeeds)):
    seed = randomSeeds[j]
    r = R2[j]
    re = MeRE[j]
    print("randomSeed :", seed)
    print("R2 Score :", round(r, 4))
    print("Median Relative Error :", round(re, 3), '%')

randomSeed : 45
R2 Score : 0.9938
Median Relative Error : 1.204 %
randomSeed : 34
R2 Score : 0.9938
Median Relative Error : 1.203 %
randomSeed : 43
R2 Score : 0.9938
Median Relative Error : 1.278 %
randomSeed : 80
R2 Score : 0.9936
Median Relative Error : 1.231 %
randomSeed : 68
R2 Score : 0.9936
Median Relative Error : 1.22 %
randomSeed : 92
R2 Score : 0.9939
Median Relative Error : 1.183 %
randomSeed : 88
R2 Score : 0.9938
Median Relative Error : 1.212 %
randomSeed : 41
R2 Score : 0.9938
Median Relative Error : 1.247 %
randomSeed : 86
R2 Score : 0.9937
Median Relative Error : 1.222 %
randomSeed : 98
R2 Score : 0.9937
Median Relative Error : 1.255 %
randomSeed : 87
R2 Score : 0.9937
Median Relative Error : 1.218 %
randomSeed : 82
R2 Score : 0.9937
Median Relative Error : 1.272 %
randomSeed : 21
R2 Score : 0.9938
Median Relative Error : 1.255 %
randomSeed : 83
R2 Score : 0.9938
Median Relative Error : 1.23 %
randomSeed : 31
R2 Score : 0.9939
Median Relative Error : 1.206 %
randomSeed :

# Performance of SigmaCCS on the external test set with different coordinates generated by ETKDG and MMFF94

In [7]:
ifile = '../data/ExternalTestData.csv'
ParameterPath = '../parameter/parameter.pkl'
mfileh5 = '../model/model_new.h5'

In [8]:
res = random.sample(range(1, 100), 30)
R2 = []
MeRE = []
randomSeeds = []

for ii in range(30):
    smiles, adduct, ccs = read_data(ifile)
    param = parameter.Parameter()
    with open(ParameterPath,'rb') as file:
        param  = pickle.loads(file.read())  
    a = res[ii]
    smiles, adduct, ccs, Coordinate = Generating_coordinates_different_seeds(smiles, adduct, ccs, a, param.All_Atoms)   ###
    
    for i in range(len(Coordinate)):
        Coordinate[i] = (np.array(Coordinate[i]) - param.Min_Coor) / (param.Max_Coor - param.Min_Coor)
    
    adj, features, edge_features = convertToGraph(smiles, Coordinate, param.All_Atoms)
    DataSet = MyDataset(features, adj, edge_features, ccs)
    
    ECC_Model = load_Model_from_file(mfileh5)
        
    re = predict(ECC_Model,param.adduct_SET,DataSet,adduct,)
    print("The number of molecules in the test set :", len(re), '\n')

    Sigma   = re
    CCS     = ccs
    SigmaPer = Metrics(CCS, Sigma)
    print("randomSeed :", a)
    print("R2 Score :", SigmaPer[0])
    print("Median Relative Error :", SigmaPer[1], '%')
    
    R2.append(SigmaPer[0])
    MeRE.append(SigmaPer[1])
    randomSeeds.append(a)

..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 343/344 [00:05<00:00, 62.32it/s]
The number of molecules in the test set : 344 

randomSeed : 97
R2 Score : 0.9792576132103041
Median Relative Error : 1.871191665827953 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 343/344 [00:05<00:00, 65.45it/s]
The number of molecules in the test set : 344 

randomSeed : 22
R2 Score : 0.9793375253036047
Median Relative Error : 1.8811994244720718 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

...

Predictions: 100%|█████████████████████████████████████▉| 343/344 [00:05<00:00, 64.42it/s]
The number of molecules in the test set : 344 

randomSeed : 13
R2 Score : 0.9797374382133336
Median Relative Error : 1.9189949113280895 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 343/344 [00:05<00:00, 64.69it/s]
The number of molecules in the test set : 344 

randomSeed : 65
R2 Score : 0.9795750322466021
Median Relative Error : 1.9842256685621396 %


..\sigma\GraphData.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray(features)
..\sigma\GraphData.py:188: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edge_features = np.asarray(edge_features)
C:\Users\yxliao\anaconda3\envs\sig\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or av

Predictions: 100%|█████████████████████████████████████▉| 343/344 [00:05<00:00, 65.08it/s]
The number of molecules in the test set : 344 

randomSeed : 62
R2 Score : 0.9793332338379265
Median Relative Error : 1.950404333516613 %


In [9]:
r2_mean = statistics.mean(R2)
r2_std = statistics.stdev(R2)

MeRE_mean = statistics.mean(MeRE)
MeRE_std = statistics.stdev(MeRE)

print("the mean value of R2 :", round(r2_mean, 4))
print("the standard deviation of R2 :", round(r2_std, 5))

print("the mean value of Median Relative Error :", round(MeRE_mean, 3), '%')
print("the standard deviation of Median Relative Error :", round(MeRE_std, 4), '%')

the mean value of R2 : 0.9796
the standard deviation of R2 : 0.00028
the mean value of Median Relative Error : 1.93 %
the standard deviation of Median Relative Error : 0.0445 %


In [10]:
for j in range(len(randomSeeds)):
    seed = randomSeeds[j]
    r = R2[j]
    re = MeRE[j]
    print("randomSeed :", seed)
    print("R2 Score :", round(r, 4))
    print("Median Relative Error :", round(re, 3), '%', '\n')

randomSeed : 97
R2 Score : 0.9793
Median Relative Error : 1.871 % 

randomSeed : 22
R2 Score : 0.9793
Median Relative Error : 1.881 % 

randomSeed : 45
R2 Score : 0.9798
Median Relative Error : 1.892 % 

randomSeed : 91
R2 Score : 0.9795
Median Relative Error : 1.891 % 

randomSeed : 78
R2 Score : 0.9799
Median Relative Error : 1.871 % 

randomSeed : 80
R2 Score : 0.9796
Median Relative Error : 1.954 % 

randomSeed : 93
R2 Score : 0.9797
Median Relative Error : 1.954 % 

randomSeed : 3
R2 Score : 0.9799
Median Relative Error : 1.878 % 

randomSeed : 50
R2 Score : 0.9793
Median Relative Error : 2.006 % 

randomSeed : 28
R2 Score : 0.9792
Median Relative Error : 1.969 % 

randomSeed : 76
R2 Score : 0.9802
Median Relative Error : 1.871 % 

randomSeed : 6
R2 Score : 0.9797
Median Relative Error : 1.976 % 

randomSeed : 15
R2 Score : 0.9793
Median Relative Error : 2.007 % 

randomSeed : 51
R2 Score : 0.9796
Median Relative Error : 1.939 % 

randomSeed : 66
R2 Score : 0.9797
Median Relative 